In [6]:
# !pip install requests beautifulsoup4 spacy transformers newspaper3k
# !python -m spacy download en_core_web_trf
!pip install spacy[transformers]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.8/758.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [1]:
import requests
import json
import re
import spacy
from bs4 import BeautifulSoup
from readability import Document
from transformers import pipeline
from urllib.parse import urlparse

In [2]:
# Load NLP models
print("Loading models...")
nlp = spacy.load("en_core_web_trf")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("sentiment-analysis")
print("Models loaded.")

Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


Models loaded.


In [30]:
def extract_author_and_date(soup):
    title_tag = soup.find('h1')
    author, date = None, None

    if title_tag:
        for sibling in title_tag.find_all_next(['div', 'span', 'p'], limit=5):
            text = sibling.get_text(separator=' ', strip=True)
            if 'By' in text and re.search(r'[A-Za-z]+ \d{1,2}, \d{4}', text):
                author_match = re.search(r'By\s*([^|]+)', text)
                if author_match:
                    author = author_match.group(1).strip()
                date_match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})', text)
                if date_match:
                    date = date_match.group(1)
                break

    if not date:
        all_text = soup.get_text()
        date_match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})', all_text)
        if date_match:
            date = date_match.group(1)
    return author, date

In [35]:
def generic_scrape(url):
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # Use readability to extract main content
    readable_doc = Document(html)
    content_html = readable_doc.summary()
    title = readable_doc.title()

    content_soup = BeautifulSoup(content_html, 'html.parser')
    content = content_soup.get_text(separator='\n', strip=True)

    # Extract image (prefer from readable content)
    image_url = None
    img_tag = content_soup.find('img')
    if img_tag and img_tag.has_attr('src'):
        image_url = img_tag['src']
    else:
        img_tag = soup.find('img')
        if img_tag and img_tag.has_attr('src'):
            image_url = img_tag['src']

    return {
        'url': url,
        'title': title,
        'content': content,
        'image_url': image_url
    }

In [36]:
def extract_metadata(article_text):
    doc = nlp(article_text)
    people, orgs, locations = set(), set(), set()

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            people.add(ent.text)
        elif ent.label_ == "ORG":
            orgs.add(ent.text)
        elif ent.label_ in ["GPE", "LOC"]:
            locations.add(ent.text)

    summary = summarizer(article_text[:1024])[0]['summary_text'] if article_text.strip() else ""
    sentiment = sentiment_analyzer(article_text[:512])[0]['label'].lower() if article_text.strip() else "neutral"

    return {
        "people_mentioned": list(people),
        "organizations": list(orgs),
        "locations": list(locations),
        "summary": summary,
        "sentiment": sentiment
    }

In [37]:
def main():
    url = input("Enter a news article URL: ").strip()
    print("Scraping article...")
    article_data = generic_scrape(url)

    # 🔽 Just add author and date extraction here
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    author, date = extract_author_and_date(soup)

    article_data['author'] = author
    article_data['date'] = date

    print("Extracting metadata...")
    metadata = extract_metadata(article_data['content'])
    article_data.update(metadata)

    # Use article title to generate filename
    title = article_data.get('title') or "article"
    title_slug = re.sub(r'[^\w\s-]', '', title)         # remove special chars
    title_slug = re.sub(r'\s+', '_', title_slug.strip()) # spaces -> underscores
    title_slug = title_slug[:60]  # limit filename length

    output_file = f"{title_slug}.json"

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump([article_data], f, ensure_ascii=False, indent=2)

    print(f"\n✅ Scraped data saved to: {output_file}")

In [39]:
if __name__ == "__main__":
    main()

Enter a news article URL: https://www.geo.tv/latest/613313-actor-humaira-asghars-body-likely-six-months-old-dig
Scraping article...
Extracting metadata...

✅ Scraped data saved to: Actor_Humaira_Asghars_body_likely_six_months_old_DIG.json
